# Исследование надежности заемщиков.

Цель исследования - необходимо выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Для этого нужно ответить на следующие вопросы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

Результаты исследования будут учтены при построении модели кредитного скоринга.

##  Обзор данных

Импортируем необходимую библиотеку `pandas`.  

In [1]:
import pandas as pd

Прочитаем файл с данными.

In [2]:
try:
    #загрузка файла по локальному пути
    data = pd.read_csv('data.csv')    
except:
    #загрузка файла по серверному пути
    data = pd.read_csv('/datasets/data.csv') 

data.head(10)     # получение первых 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Описание данных:
`children` — количество детей в семье
`days_employed` — общий трудовой стаж в днях
`dob_years` — возраст клиента в годах
`education` — уровень образования клиента
`education_id` — идентификатор уровня образования
`family_status` — семейное положение
`family_status_id` — идентификатор семейного положения
`gender` — пол клиента
`income_type` — тип занятости
`debt` — имел ли задолженность по возврату кредитов
`total_income` — ежемесячный доход
`purpose` — цель получения кредита

Изучим общую информацию о таблице

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Названия столбцов таблицы соответствуют общим правилам стиля и не требуют изменений. В столбцах разичается количество значений, что свидетельствует о наличии пропущенных значений. Таким образом пропущенные значния присутствуют с стобцах "days_employed", "total_income". Типы всех столбцов соответствуют информации хранящихся в них.   

## Проверка на пропущенные значения

Посчитаем пропущенные значения

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Одинаковое количество пропущенных значений в столбцах содержащих информацию об общем трудовом стаже в днях и ежемесячном доходе. Посмотрим на строки с пропущенными значениями 

In [5]:
data[data['days_employed'].isna()].head(20)   

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


В строках где пропущено значение в столбце 'days_employed' отсутствует знвчение и в столбце 'total_income', таких значений 2174. Проверим это посчитав пропущенные значения в столбце total_income для строк в которых пропущены значения в столбце days_employed.  


In [7]:
data[data['days_employed'].isna()]['total_income'].isna().sum()  

2174

Посчитаем долю пропущенных значений в этих столбцах от общего количества значений.

In [8]:
data['days_employed'].isna().mean()

0.10099883855981417

Доля пропущенных значений в столбце  'days_employed' составляет чуть более 0,1.

In [9]:
data['total_income'].isna().mean()

0.10099883855981417

Доля пропущенных значений в столбце 'total_income' также составляет чуть более 0.1. Что достаточно много для того что бы удалить их. Поэтому пропущенные значения в данных столбцах можно будет заполнить медианным значением. Но перед этим необходимо удалить все дубликаты.
___
Значения пропущенны с столбцах 'total_income' и 'days_employed' одновременно  Данные пропуски могли возникнуть из-за технической ошибки, а возможно люди не указывали свой доход и стаж, так как у них отсутствует официальный заработок и стаж и соответственно не было документов оснований для заполнения данных граф. 

Посчитаем медианное значение для столбцов 'total_income' и 'days_employed'

In [10]:
data['days_employed'].median()  #посчитаем медиану

-1203.369528770489

In [11]:
data['total_income'].median()   #посчитаем медиану

145017.93753253992

Медианное значение для столбца 'days_employed' отрицательное, что противоречит смыслу содержащихся в данном столбце значений. Таким образом, для корректного заполнения пропущенных значений в данном столбце необходимо сначала избавиться от аномалий в значениях.

## Проверка данных на аномалии

Проверим на корректность значения в столбцах. Для этого напишем функцию которая выведет максимальное и минимальное значение в столбце. И проверим столбцы, которые содержат числовые значения (данные типа int64 и float64)  

In [12]:
def max_min(columns):
    for column in columns:
        print(column, max(data[column]), min(data[column]))

max_min (['children','days_employed', 'dob_years', 'education_id', 'family_status_id', 'debt','total_income'])


children 20 -1
days_employed 401755.40047533 -18388.949900568383
dob_years 75 0
education_id 4 0
family_status_id 4 0
debt 1 0
total_income 2265604.028722744 20667.26379327158


 В столбце children присутствует отрицательное значение, что противоречит смыслу данного столбца (количество детей не может быть отрицательным). Проверим сколько уникальных значений в данном столбце и их количество.

In [13]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Отрицательное значение в данном столбце одно и встречается 47 раз. Логично заменить его на 1. Также 20 детей кажется слишком большим значением.
Посчитаем долю значений -1 в общем количестве значений

In [14]:
len(data[data['children'] == -1]) / len(data['children'])

0.002183507549361208

Значения -1 составляют чуть более 0.2% от общей массы. Замена значений -1 на 1 не внесет значительных изменений.

In [15]:
data['children'] = data['children'].replace(-1, 1)
data['children'].value_counts()     # проверим уникальные значения после замены

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Посчитаем долю значений 20 в общем количестве значений

In [16]:
len(data[data['children'] == 20]) / len(data['children'])

0.0035307781649245064

Доля значений равных 20 составляет чуть более 0,35%. Так как других значений с большим количеством детей нет(от 5 до 20 нет другх значений), то вероятно это аномалия, а не реальное значение. Возможно данная аномалия вызвана опечаткой и на самом деле значение должно быть равно 2 или 0. В случае со значением -1 считаю, что ввести вместо 0 значение 1 наименее вероятно и появление "-" в данном столбце ошибка, в то время как со значением 20 возможно больше вариантов и так как доля данных значений невелика их можно удалить.

In [17]:
# удалим стороки со значением 20
data = data[data['children'] != 20]

Проверим уникальные значения после изменений

In [18]:
data['children'].value_counts()

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Посмотрим общую информацию по датасету после изменений

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 12 columns):
children            21449 non-null int64
days_employed       19284 non-null float64
dob_years           21449 non-null int64
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        19284 non-null float64
purpose             21449 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Также в столбце days_employed присуствуют отрицательные значения. Стаж не может быть отрицательным, в сзязи с чем логично заменить данные значения на положительные. Данная ошибка могла возникнуть из-за того что даты для подсчета стажа были перепутаны местами и из более раннней даты вычиталась более поздняя. Также максимальное значение в данной столбце значительно превышает максимально возможный рабочий стаж. 
Посчитаем максимальный стаж из наших данных в годах, поделим на 365 дней в году.

In [20]:
401755.40047533 / 365

1100.6997273296713

И хотя не все дни в году являются рабочими, но даже при делении на 365 получается слишком большое количество лет. Возможно данные в данной столбце не в днях и тогда стоит более тщательно изучить данные в этом столбце, но в проверяемой гипотезе нет вопроса о стаже, в связи с чем не корректные данные в столбце можно проигнорировать.
Заменим отрицательные данные на положительные, чтобы затем корректно посчитать медианное значение.

In [21]:
data['days_employed'] = data['days_employed'].abs()

Проверим наличие отрицательных и равных нулю занчений в столбце 'days_employed'

In [22]:
data[data['days_employed']<= 0]['days_employed'].count()

0

В столбце dob_years минимальное значение 0. Получается некоторые люди просто не указали свой возраст. Проверим какие еще значения есть в данном столбце. Для этого посмотрим на уникальные значения данного столбца.  

In [23]:
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

Остальные занчения соответсвуют реальности, таким образом анаомальным является значение 0 в данном столбце. Но так как зависимость от возраста мы не рассматриваем, то можно его не изменять.   

Проверим столбец gender на наличие аномалий в данных.

In [24]:
data['gender'].value_counts()

F      14189
M       7259
XNA        1
Name: gender, dtype: int64

Столбец содержит некорректное значения с полом XNA. Данное значение встечается только один раз и строку можно удалить, но так как в проверяемой гипотезе нет вопроса о зависимости от поле, то не будем удалять строку.   

##  Удаление дубликатов

Проверим датасет на наличие явных дубликатов и сразу посчитаем их

In [25]:
data.duplicated().sum()

54

Удалим явных дубликатов с удалением старых индексов и формированием новых

In [26]:
data = data.drop_duplicates().reset_index(drop = True)

Ещё раз посчитаем явные дубликаты в таблице, чтобы убедиться, что полностью от них избавились

In [27]:
data.duplicated().sum()

0

Теперь проверим датасет на наличие неявных дубликатов. Начнем проверку со столбца education.

In [28]:
data['education'].value_counts() # посчитаем сколько уникальных значений и какие

среднее                13653
высшее                  4698
СРЕДНЕЕ                  770
Среднее                  705
неоконченное высшее      666
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Значения столбца повторяются, но написанные разным регистром. Необходимо привести написание к единому регистру ля удаления неявных дубликатов.

In [29]:
data['education'] = data['education'].str.lower() # приведим значения столбца к нижнему регистру
data['education'].value_counts()                  # пересчитаем сколько уникальных значений и какие

среднее                15128
высшее                  5237
неоконченное высшее      742
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь проверим столбец family_status на наличие неявных дубликатов. 

In [30]:
data['family_status'].value_counts()

женат / замужем          12295
гражданский брак          4151
Не женат / не замужем     2801
в разводе                 1193
вдовец / вдова             955
Name: family_status, dtype: int64

Неявные дубликаты в столбце family_status  отсутствуют.

Проверим столбец income_type на наличие неявных дубликатов.

In [31]:
data['income_type'].value_counts()

сотрудник          11048
компаньон           5058
пенсионер           3828
госслужащий         1455
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Неявные дубликаты в столбце income_type отсутствуют.

Проверим столбец purpose на наличие неявных дубликатов.

In [32]:
data['purpose'].value_counts()

свадьба                                   792
на проведение свадьбы                     769
сыграть свадьбу                           765
операции с недвижимостью                  674
покупка коммерческой недвижимости         659
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
покупка жилья                             643
жилье                                     642
покупка жилья для семьи                   637
недвижимость                              632
строительство собственной недвижимости    629
операции со своей недвижимостью           626
строительство жилой недвижимости          624
покупка своего жилья                      620
строительство недвижимости                619
покупка недвижимости                      619
ремонт жилью                              605
покупка жилой недвижимости                604
на покупку своего автомобиля              505
заняться высшим образованием      

Неявные дубликаты в столбце purpose отсутствуют. Все значения записаны в одном регистре. Все значения в данном столбце сформулированы по разному, но их можно разделить на 4 категории: ‘операции с автомобилем’, ‘операции с недвижимостью’, ‘проведение свадьбы’, ‘получение образования’. Категоризацию целей кредита произвелем далее. 

После удаления неявных дубликатов необходимо повторно проверить датасет на наличие явных дубликатов.

In [33]:
data.duplicated().sum() # посчитаем количество явных дубликатов

17

Удалим строки с явными дубликатами.

In [34]:
data = data.drop_duplicates().reset_index(drop = True)   # удалим строки с явными дубликатами
data.duplicated().sum()                                  # посчитаем количество явных дубликатов

0

Посмотрим общую информацию по датасету после удаления дубликатов.

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21378 entries, 0 to 21377
Data columns (total 12 columns):
children            21378 non-null int64
days_employed       19284 non-null float64
dob_years           21378 non-null int64
education           21378 non-null object
education_id        21378 non-null int64
family_status       21378 non-null object
family_status_id    21378 non-null int64
gender              21378 non-null object
income_type         21378 non-null object
debt                21378 non-null int64
total_income        19284 non-null float64
purpose             21378 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Заполнение пропусков

После удаления дубликатов вернемся к заполнению пропущенных значений в столбцах  days_employed и total_income. Теперь мы избавились от отрицательных значений в столбце days_employed и дубликатов, что позволит более точно рассчитать медианное значение для заполнения пропущенных значений. Рассчитаем значения медианы для столбца days_employed.

In [36]:
median_days_employted = data['days_employed'].median()  #посчитаем медиану
median_days_employted

2196.507938101055

В проверяемой гипотезе нет вопроса о стаже, в связи с чем не корректные данные в столбце можно проигнорировать. Заменим пропущенные значения на полученное медианное значение.

In [37]:
data['days_employed'] = data['days_employed'].fillna(median_days_employted) # заменяем значения в столбце days_employed

Рассчитаем значения медианы для столбца total_income.

In [38]:
median_total_income = data['total_income'].median()    #посчитаем медиану
median_total_income

145020.80127962783

В датаcете есть столбец income_type (тип занятости), возможно, что ежемесячный доход зависит от типа занятости. Посмотрим на количество значений и на медианное значение для каждого типа занятости. Для этого построим сводную таблицу.

In [39]:
data.pivot_table(index='income_type', values='total_income', aggfunc=['median', 'count'])

,median,count
,total_income,total_income
income_type,,
безработный,131339.751676,2
в декрете,53829.130729,1
госслужащий,150420.150276,1311
компаньон,172450.279990,4558
пенсионер,118406.283533,3436
предприниматель,499163.144947,1
сотрудник,142594.396847,9974
студент,98201.625314,1


Таким образом, видим что в некоторых категориях только одно или два значения и медианное значение не имеет смысла.Посмотрим для какого типа занятости пропущены значения в столбце total_income

In [40]:
data[data['total_income'].isna()]['income_type'].value_counts()

сотрудник          1067
компаньон           498
пенсионер           384
госслужащий         144
предприниматель       1
Name: income_type, dtype: int64

Видим, что пропущенные значения присутствуют только в 5 типах занятости, при этом для типа занятости "предприниматель" только 1 значение и расчет медианного не имеет смысла и ориентироваться на одно значение в категории не верно. Для остальных 4 типов медианные значения как видно из сводной таблицы выше находятся в диапазоне от 118 тыс. (пенсионер) до 172 тыс. (компаньон), что соответствует категории "С", также как и медианное значение для всего столбца total_income (145 тыс.). Категоризация доходов производится далее, но данном этапе видно, что можно заполнить пропущенные значения в столбце total_income медианным значением не исказив данные.  

Заменим пропущенные значения на медиану.

In [41]:
data['total_income'] = data['total_income'].fillna(median_total_income)     # заменяем значения в столбце total_income
data.head(10)                                                               # выведем первые 10 строк 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Посчитаем пропущенные значения после заполнения.

In [42]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропущенных значений теперь нет.

## Изменение типов данных.

Выведем общую информацию о датасете

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21378 entries, 0 to 21377
Data columns (total 12 columns):
children            21378 non-null int64
days_employed       21378 non-null float64
dob_years           21378 non-null int64
education           21378 non-null object
education_id        21378 non-null int64
family_status       21378 non-null object
family_status_id    21378 non-null int64
gender              21378 non-null object
income_type         21378 non-null object
debt                21378 non-null int64
total_income        21378 non-null float64
purpose             21378 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Во всех столбцах содержится одинаковое количество значений, пропущенных значений нет, все данные в подходящем типе. 
Для удобства и более высокой точности приведем целочисленные значения в столбце total_income к типу int.

In [44]:
data['total_income'] = data['total_income'].astype('int')
data['total_income']

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21373    224791
21374    155999
21375     89672
21376    244093
21377     82047
Name: total_income, Length: 21378, dtype: int64

Проверим изменение типа данных по общей информации

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21378 entries, 0 to 21377
Data columns (total 12 columns):
children            21378 non-null int64
days_employed       21378 non-null float64
dob_years           21378 non-null int64
education           21378 non-null object
education_id        21378 non-null int64
family_status       21378 non-null object
family_status_id    21378 non-null int64
gender              21378 non-null object
income_type         21378 non-null object
debt                21378 non-null int64
total_income        21378 non-null int64
purpose             21378 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма со столбцами:
education_id и education — в первом;
family_status_id и family_status — во втором.
Эти новые датафреймы будут «словарями», к которым можно будет обращаться по идентификатору.

Создадим датафрейм с данными об образовании.

In [46]:
education_dict = data[['education', 'education_id']]
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21373,среднее,1
21374,среднее,1
21375,среднее,1
21376,среднее,1


В полученном датафрейме много дубликатов. Посчитаем их.

In [47]:
education_dict.duplicated().sum()

21373

Удалим дубликаты и изменим индексацию, выведем полученный датафрейм.


In [48]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Теперь в датафрейме нет дубликатов и каждому education_id соответствует свое значение.

Создадим датафрейм с данными о семейном положении и посмотрим на полученный результат.

In [49]:
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
...,...,...
21373,гражданский брак,1
21374,женат / замужем,0
21375,гражданский брак,1
21376,женат / замужем,0


Удалим дубликаты и изменим индексацию, выведем полученный датафрейм.

In [50]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Теперь в датафрейме нет дубликатов и каждому family_status_id соответствует свое значение.

Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. 

In [51]:
data = data.drop(columns = ['family_status', 'education'],axis = 1) 
data.head()             # выведем первые 5 строк

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


## Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.
Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.

Напишем функцию которая будет возвращать нужную категорию в соотвествии со значением дохода.

In [52]:
def income_category(income):
    try:
        if 0 < income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        return print("Доход не может быть отрицательным")

Добвим в датафрейм столбец total_income_category с категориями. Для этого применим созданную функцию.

In [53]:
data['total_income_category'] = data['total_income'].apply(income_category)
data.head()      # выведем первые 5 строк

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Посмотрим на уникальные значения нового столбца

In [54]:
data['total_income_category'].value_counts()

C    15960
B     5021
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Проверим все ли значения в новом столбце заполнены по общей информации. 

In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21378 entries, 0 to 21377
Data columns (total 11 columns):
children                 21378 non-null int64
days_employed            21378 non-null float64
dob_years                21378 non-null int64
education_id             21378 non-null int64
family_status_id         21378 non-null int64
gender                   21378 non-null object
income_type              21378 non-null object
debt                     21378 non-null int64
total_income             21378 non-null int64
purpose                  21378 non-null object
total_income_category    21378 non-null object
dtypes: float64(1), int64(6), object(4)
memory usage: 1.8+ MB


Пустых значений в новой столбце нет.
Таким образом, произведена категоризация данных по уровню дохода.Теперь основываясь на категории возможно изучить влияет ли какая-то категория на вероятность задолженности по кредиту.

## Категоризация целей кредита.

Мы видели, что в столбце purpose находились дублирующие причины, которые отличались формулировкой, поэтому причины стоит категоризировать. 
Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
‘операции с автомобилем’,
‘операции с недвижимостью’,
‘проведение свадьбы’,
‘получение образования’.
Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.
Вы можете использовать собственную функцию и метод apply(). Изучите данные в столбце purpose и определите, какие подстроки помогут вам правильно определить категорию

Еще раз выведем все уникальные значения столбца purpose

In [56]:
data['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     764
сыграть свадьбу                           761
операции с недвижимостью                  674
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   650
операции с жильем                         648
операции с коммерческой недвижимостью     646
покупка жилья                             643
жилье                                     642
покупка жилья для семьи                   637
недвижимость                              632
строительство собственной недвижимости    629
операции со своей недвижимостью           626
строительство жилой недвижимости          623
покупка своего жилья                      620
строительство недвижимости                619
покупка недвижимости                      619
ремонт жилью                              605
покупка жилой недвижимости                604
на покупку своего автомобиля              505
заняться высшим образованием      

Для категоризации целей кредита используем лемматизацию. Для этого импортируем необходимую библиотеку 

In [57]:
from pymystem3 import Mystem
m = Mystem()

В сзязи с тем что библиотека pymystem3 на операционной системе Windows работает крайне долго, то лемматизацию применим только к уникальным значеням столбца purpose. 
Создадим словарь main_category который будет содержать в качестве ключа - лемму ключевого слова по которой будет происходить определение категории, а в качестве значения - категорию целей кредита. Проанализировав уникальные значения столбца purpose в качестве леммы ключевого слова примимаем следующие значения: свадьба, жилье, недвижимость, автомобиль. А заполнять будем словарь purpose_category который в качестве ключа будет содержать уникальные значения столбца purpose, а в качества значения - категорию целей кредита. 

In [58]:
purpose_category = {}  # создаем пустой словарь для записи 
main_category = {'свадьба':'проведение свадьбы', 'жилье':'операции с недвижимостью', 
                 'недвижимость':'операции с недвижимостью', 'автомобиль':'операции с автомобилем', 
                 'образование':'получение образования'} # создаем словарь с ключевыми словами и категориями целей кредита
purpose_unique = list(data['purpose'].unique()) # создаем список уникальных значений столбца purpose

for i in purpose_unique:            # для каждого уникальных значений столбца purpose
    purpose_category[i] = ''        # в словарь purpose_category записываем в качестве ключа уникальное значение столбца purpose
    lemmas = m.lemmatize(i)         # применяем лемматизацию к каждому уникальному значению столбца purpose
    for word in lemmas:             # для каждого слова из леммы из уникального значения столбца purpose
        if word in main_category.keys(): # проверяем соотвествует ли слово одному ключей словаря main_category
            purpose_category[i] = main_category[word] # если соотвествует присваеваем значение катагории в словарь purpose_category 
    if purpose_category[i] == '':        # если значение словаря purpose_category не заполнилось в соответствии с main_category
        purpose_category[i] = 'прочие цели' # то присваиваем категорию "прочие цели"

purpose_category # выведем созданный словарь


{'покупка жилья': 'операции с недвижимостью',
 'приобретение автомобиля': 'операции с автомобилем',
 'дополнительное образование': 'получение образования',
 'сыграть свадьбу': 'проведение свадьбы',
 'операции с жильем': 'операции с недвижимостью',
 'образование': 'получение образования',
 'на проведение свадьбы': 'проведение свадьбы',
 'покупка жилья для семьи': 'операции с недвижимостью',
 'покупка недвижимости': 'операции с недвижимостью',
 'покупка коммерческой недвижимости': 'операции с недвижимостью',
 'покупка жилой недвижимости': 'операции с недвижимостью',
 'строительство собственной недвижимости': 'операции с недвижимостью',
 'недвижимость': 'операции с недвижимостью',
 'строительство недвижимости': 'операции с недвижимостью',
 'на покупку подержанного автомобиля': 'операции с автомобилем',
 'на покупку своего автомобиля': 'операции с автомобилем',
 'операции с коммерческой недвижимостью': 'операции с недвижимостью',
 'строительство жилой недвижимости': 'операции с недвижимост

Напишем функцию которая вернет значение категории по значению столбца purpose.

In [59]:
def category_purpose_debt(purpose):
    return purpose_category[purpose]

Сформируем новый столбец purpose_category, в который войдут следующие категории целей кредита.
Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.
Для этого используем функцию category_purpose_debt и метод apply(). 

In [60]:
data['purpose_category'] = data['purpose'].apply(category_purpose_debt)
data.head(10) # выведем первые 10 строк

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


Посмотрим на уникальные значения нового столбца

In [61]:
data['purpose_category'].value_counts()

операции с недвижимостью    10775
операции с автомобилем       4290
получение образования        3998
проведение свадьбы           2315
Name: purpose_category, dtype: int64

Все распределено по заданным категориям, значений в категории "прочие цели" нет. 

Проверим все ли значения в новом столбце заполнены по общей информации.

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21378 entries, 0 to 21377
Data columns (total 12 columns):
children                 21378 non-null int64
days_employed            21378 non-null float64
dob_years                21378 non-null int64
education_id             21378 non-null int64
family_status_id         21378 non-null int64
gender                   21378 non-null object
income_type              21378 non-null object
debt                     21378 non-null int64
total_income             21378 non-null int64
purpose                  21378 non-null object
total_income_category    21378 non-null object
purpose_category         21378 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Пустых значений в новом столбце нет. Таким образом, произведена категоризация данных по целям кредита.Теперь основываясь на категории возможно изучить влияет ли какая-то категория на вероятность задолженности по кредиту.

## Ответы на вопросы.

### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?
Есть ли зависимость между семейным положением и возвратом кредита в срок?
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Как разные цели кредита влияют на его возврат в срок?

Для ответа на данный вопрос сгруппируем данные по количеству детей и посчитаем количество кредитов в группе и количеством невозвратов в срок и  долю невозвратов в каждой группе. 

In [63]:
data_of_children = data.groupby(['children']).agg({'debt':['count', 'sum', 'mean']})
data_of_children

debt                
          count   sum      mean
children                       
0         14091  1063  0.075438
1          4855   445  0.091658
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000

Таким образом, в таблице наглядно видно что доля невозвратов в срок меньше всего у людей без детей. С увеличением количества детей доля невозвратов в срок растет, за исключением клиентов с 3 детьми - доля невозвратов уменьшилась, но все равно больше чем у клиентов без детей, и клиентов с 5 детьми - невозвраты отсутствуют, но  и кретов всего 9 штук.

### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Как разные цели кредита влияют на его возврат в срок?

Для ответа на данный вопрос сгруппируем данные по семейному положению и посчитаем количество кредитов кредитов в группе и количеством невозвратов в срок. 

In [64]:
data_of_family = data.groupby('family_status_id').agg({'debt':['count', 'sum']})
data_of_family

debt     
                  count  sum
family_status_id            
0                 12290  928
1                  4139  385
2                   955   63
3                  1193   84
4                  2801  273

Для корректной оценки посчитаем долю невозвратов в каждой группе и добавим столбец overdue_ration с этими значениями в таблицу

In [65]:
data_of_family['overdue_ration'] = data_of_family['debt']['sum'] / data_of_family['debt']['count']
data_of_family

debt      overdue_ration
                  count  sum               
family_status_id                           
0                 12290  928       0.075509
1                  4139  385       0.093018
2                   955   63       0.065969
3                  1193   84       0.070411
4                  2801  273       0.097465

Добавим значения столбца семейного положения соответствующие столбцу family_status_id. Для этого объединим таблицу data_of_family с таблицей family_status_dict по столбцу family_status_id 

In [66]:
data_of_family = data_of_family.merge(family_status_dict, on='family_status_id', how='right')
data_of_family

/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,family_status_id,"(debt, count)","(debt, sum)","(overdue_ration, )",family_status
0,0,12290,928,0.075509,женат / замужем
1,1,4139,385,0.093018,гражданский брак
2,2,955,63,0.065969,вдовец / вдова
3,3,1193,84,0.070411,в разводе
4,4,2801,273,0.097465,Не женат / не замужем


Таким образом, в таблице наглядно видно что доля невозвратов в срок меньше всего у вдовцов/вдов, а больше всего у клиентов в статусе Не женат / не замужем, а также в стасусе Гражданский брак.  

### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Как разные цели кредита влияют на его возврат в срок?

Для ответа на данный вопрос сгруппируем данные по категории уровня дохода и посчитаем количество кредитов кредитов в группе и количеством невозвратов в срок. 

In [67]:
data_of_income = data.groupby('total_income_category').agg({'debt':['count', 'sum']})
data_of_income

debt      
                       count   sum
total_income_category             
A                         25     2
B                       5021   354
C                      15960  1354
D                        350    21
E                         22     2

Для корректной оценки посчитаем долю невозвратов в каждой группе и добавим столбец overdue_ration с этими значениями в таблицу

In [68]:
data_of_income['overdue_ration'] = data_of_income['debt']['sum'] / data_of_income['debt']['count']
data_of_income

debt       overdue_ration
                       count   sum               
total_income_category                            
A                         25     2       0.080000
B                       5021   354       0.070504
C                      15960  1354       0.084837
D                        350    21       0.060000
E                         22     2       0.090909

Таким образом, в таблице наглядно видно что доля невозвратов в срок меньше всего у клиентов в категории дохода D, а больше всего в категории дохода Е. Зависимости что с ростом уровня дохода снижается количество невозвратов нет, так как в категории доходов А и Е доля невозвратов довольно высока. 

### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

Для ответа на данный вопрос сгруппируем данные по категории целей кредита и посчитаем количество кредитов кредитов в группе и количеством невозвратов в срок. 

In [69]:
data_of_purpose = data.groupby('purpose_category').agg({'debt':['count', 'sum']})
data_of_purpose

debt     
                          count  sum
purpose_category                    
операции с автомобилем     4290  401
операции с недвижимостью  10775  780
получение образования      3998  369
проведение свадьбы         2315  183

Для корректной оценки посчитаем долю невозвратов в каждой группе и добавим столбец overdue_ration с этими значениями в таблицу

In [70]:
data_of_purpose['overdue_ration'] = data_of_purpose['debt']['sum'] / data_of_purpose['debt']['count'] 
data_of_purpose

debt      overdue_ration
                          count  sum               
purpose_category                                   
операции с автомобилем     4290  401       0.093473
операции с недвижимостью  10775  780       0.072390
получение образования      3998  369       0.092296
проведение свадьбы         2315  183       0.079050

Таким образом, в таблице наглядно видно что доля невозвратов в срок меньше всего по операциям c недвижимостью, а больше всего по операциям с автомобилем. Но в целом доля невозвратов не сильно изменяется в зависимости от цели кредита. 

## Общий вывод:

Исходя из полученных данных можно сделать следующий вывод: Семейное положение и наличие детей в семье влияют на факт погашения кредита в срок. Так доля невозвратов в срок меньше всего у людей без детей и в целом с увеличением количества детей доля невозвратов в срок растет. Также доля невозвратов в срок меньше всего у вдовцов/вдов, а больше всего у клиентов в статусе Не женат / не замужем, а также в стасусе Гражданский брак. 
Доля невозвратов в срок меньше всего у клиентов в категории дохода D, а больше всего в категории дохода Е. Зависимости что с ростом уровня дохода снижается количество невозвратов нет, так как в категории доходов А и Е доля невозвратов довольно высока. 
Доля невозвратов в срок меньше всего по операциям c недвижимостью, а больше всего по операциям с автомобилем и на получение образования. Но в целом доля невозвратов не сильно изменяется в зависимости от цели кредита. 
Поэтому наиболее надежными клиентами для банка являются вдовы/вдовцы без детей или с небольшим количеством детей с категорией дохода D и с целью кредита на операции c недвижимостью.